## Load Model

In [ ]:
!pip uninstall adapter-transformers

In [ ]:
# from transformers import AutoModelWithHeads

# model = AutoModelWithHeads.from_pretrained("bert-base-uncased")
# adapter_name = model.load_adapter("AdapterHub/bert-base-uncased-pf-conll2003", source="hf")
# model.active_adapters = adapter_name

In [ ]:
# from transformers import pipeline
# nlp = pipeline("ner", model=model, tokenizer=tokenizer)
# example = "My name is Wolfgang and I live in Berlin"

In [58]:
### USEFUL MODEL 1

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("philschmid/distilroberta-base-ner-conll2003")
model1 = AutoModelForTokenClassification.from_pretrained("philschmid/distilroberta-base-ner-conll2003")

model1_nlp = pipeline("ner", model=model1, tokenizer=tokenizer, grouped_entities=True)
example = "My name is Philipp and live in Germany"

model1_nlp(example)

/Users/nanfangwuyu/opt/anaconda3/envs/Classifier/lib/python3.9/site-packages/transformers/pipelines/token_classification.py:135: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.99267524,
  'word': ' Philipp',
  'start': 11,
  'end': 18},
 {'entity_group': 'LOC',
  'score': 0.99980456,
  'word': ' Germany',
  'start': 31,
  'end': 38}]

In [31]:
# import os
# # os.chdir('InstructionNER')
# from instruction_ner.model import Model
# model = Model(
#     model_path_or_name="olgaduchovny/t5-base-ner-mit-movie",
#     tokenizer_path_or_name="olgaduchovny/t5-base-ner-mit-movie"
# )
# options = [
#         "ACTOR",
#         "AWARD",
#         "CHARACTER",
#         "DIRECTOR",
#         "GENRE",
#         "OPINION",
#         "ORIGIN",
#         "PLOT",
#         "QUOTE",
#         "RELATIONSHIP",
#         "SOUNDTRACK",
#         "YEAR"
#     ]
# instruction = "please extract entities and their types from the input sentence, " \
#               "all entity types are in options"
# text = "are there any good romantic comedies out right now"
# generation_kwargs = {
#     "num_beams": 2,
#     "max_length": 128
# }
# pred_spans = model.predict(
#     text=text,
#     generation_kwargs=generation_kwargs,
#     instruction=instruction,
#     options=options
# )

In [32]:
# model.predict(
#     text=text,
#     generation_kwargs=generation_kwargs,
#     instruction=instruction,
#     options=options
# )

('good is a REVIEW, romantic comedies is a GENRE.', [(19, 36, 'GENRE')])

In [1]:
### USEFUL MODEL 2

from transformers import pipeline
model2_nlp = pipeline('ner', model='dbmdz/bert-large-cased-finetuned-conll03-english')
message = 'Show me a picture of Halle Berry.'
entities = model2_nlp(message, aggregation_strategy="simple")
# name = ''
# for entity in entities:
#     name = name + entity['word']
#     # print(f"{entity['word']}: {entity['entity_group']} ({entity['score']:.2f})")
# name = name.replace('#','')
# print(name)

/Users/nanfangwuyu/opt/anaconda3/envs/Classifier/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

: 

In [1]:
# print(entities)

In [53]:
## Sent Sim

from sentence_transformers import SentenceTransformer, util

sent_sim = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [81]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model3 = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

model3_nlp = pipeline("ner", model=model3, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = model3_nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


## Load Data

In [9]:
qs_origin = []
Q_ents = []


In [10]:
qs_origin.extend([
    "Who is the director of Good Will Hunting?",
    "Who directed The Bridge on the River Kwai?",
    "Who is the director of Star Wars: Episode VI - Return of the Jedi?",
])

Q_ents.extend([
    "Good Will Hunting",
    "The Bridge on the River Kwai",
    "Star Wars: Episode VI - Return of the Jedi",
]) # should be extract by model

qs_origin.extend([
    "Who is the screenwriter of The Masked Gang: Cyprus?",
    "What is the MPAA film rating of Weathering with You?",
    "What is the genre of Good Neighbors?",
])

Q_ents.extend([
    "The Masked Gang: Cyprus",
    "Weathering with You",
    "Good Neighbors",
])

qs_origin.extend([
    "What is the box office of The Princess and the Frog?",
    "Can you tell me the publication date of Tom Meets Zizou?",
    "Who is the executive producer of X-Men: First Class?",
])

Q_ents.extend([
    "The Princess and the Frog",
    "Tom Meets Zizou",
    "X-Men: First Class",
])

qs_origin.extend([
    "Show me a picture of Halle Berry.",
    "What does Julia Roberts look like?",
    "Let me know what Sandra Bullock looks like.",
])

Q_ents.extend([
    "Halle Berry",
    "Julia Roberts",
    "Sandra Bullock",
])# TODO if many ents a question

In [56]:
import pandas as pd

Name_Qwiki_Qid = pd.read_csv('name_Qwiki_Qid.tsv', sep='\t')
Name_Qwiki_Qid[-5:]

names = Name_Qwiki_Qid['Str']

In [59]:
name_embeddings = sent_sim.encode(names)

In [63]:
type(name_embeddings)

numpy.ndarray

In [64]:
import numpy as np
np.save("name_embeddings.npy", name_embeddings)

## Load Test

### Test for Model 1

In [77]:


acc = 0
for i, q in enumerate(qs_origin):
    preds = model1_nlp(q, aggregation_strategy="simple")
    ents = []
    for pred in preds:
        ent = pred['word']
        if ent[0] == ' ':
            ent = ent[1:]
        ents.append(ent)
    print("--------------------------------------------")
    print("Ground truth: {}".format(Q_ents[i]))
    print("Predicted name entity: ", ents)
    

--------------------------------------------
Ground truth: Good Will Hunting
Predicted name entity:  ['Good Will Hunting']
--------------------------------------------
Ground truth: The Bridge on the River Kwai
Predicted name entity:  ['The Bridge', 'River Kwai']
--------------------------------------------
Ground truth: Star Wars: Episode VI - Return of the Jedi
Predicted name entity:  ['Star Wars: Episode VI', 'Return of the Jedi']
--------------------------------------------
Ground truth: The Masked Gang: Cyprus
Predicted name entity:  ['The', 'Masked Gang:', 'Cyprus']
--------------------------------------------
Ground truth: Weathering with You
Predicted name entity:  ['MP', 'AA', 'Weather', 'ing', 'with You']
--------------------------------------------
Ground truth: Good Neighbors
Predicted name entity:  ['Good Neighbors']
--------------------------------------------
Ground truth: The Princess and the Frog
Predicted name entity:  ['The Princess and the Frog']
-------------------

### Test for Model 2

In [91]:
acc = 0
all_ents = []
for i, q in enumerate(qs_origin):
    preds = model2_nlp(q, aggregation_strategy="simple")
    ents = []
    for pred in preds:
        ent = pred['word']
        if ent[0] == ' ':
            ent = ent[1:]
        ents.append(ent)
    print("--------------------------------------------")
    print("Ground truth: {}".format(Q_ents[i]))
    print("Predicted name entity: ", ents)
    all_ents.append(ents)


--------------------------------------------
Ground truth: Good Will Hunting
Predicted name entity:  ['Good Will Hunting']
--------------------------------------------
Ground truth: The Bridge on the River Kwai
Predicted name entity:  ['The Bridge on the River Kwai']
--------------------------------------------
Ground truth: Star Wars: Episode VI - Return of the Jedi
Predicted name entity:  ['Star Wars', 'Episode VI', 'Return of the Jedi']
--------------------------------------------
Ground truth: The Masked Gang: Cyprus
Predicted name entity:  ['The Masked Gang : Cyprus']
--------------------------------------------
Ground truth: Weathering with You
Predicted name entity:  ['MPAA', 'Weathering with You']
--------------------------------------------
Ground truth: Good Neighbors
Predicted name entity:  ['Good Nei', '##bors']
--------------------------------------------
Ground truth: The Princess and the Frog
Predicted name entity:  ['The Princess and the Frog']
-------------------------

In [93]:
all_ents = [' '.join(ents) for ents in all_ents]

#### try sent_sim for preds and whole sents compared to true Q

In [94]:
pred2_embeddings = sent_sim.encode(all_ents)

In [62]:
from sentence_transformers import util

In [98]:

matchings = []
for i, embd in enumerate(pred2_embeddings[:1]):
    results = [util.pytorch_cos_sim(embd, name_embd) for name_embd in name_embeddings]
    matching = names[results.index(max(results))]
    matchings.append(matching)
    print("X: ", all_ents[i])
    print("y: ", matching)

X:  Good Will Hunting
y:  Good Will Hunting


In [ ]:
import torch
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

dtype = torch.float
device = torch.device("mps")
# torch.nn.functional.cosine_similarity(torch.tensor(pred2_embeddings[0]), torch.tensor(name_embeddings[0]))

In [ ]:
# Here with GPU is even slower
# util.pytorch_cos_sim(torch.tensor(pred2_embeddings, device=device), torch.tensor(name_embeddings, device=device))

In [114]:
sims = util.pytorch_cos_sim(torch.tensor(pred2_embeddings), torch.tensor(name_embeddings))

In [115]:
len(pred2_embeddings)

12

In [122]:
idx = torch.max(sims, 1).indices

In [126]:
for i, id in enumerate(idx):
    print("X: ", all_ents[i])
    print("y: ", names[int(id)])

X:  Good Will Hunting
y:  Good Will Hunting
X:  The Bridge on the River Kwai
y:  The Bridge on the River Kwai
X:  Star Wars Episode VI Return of the Jedi
y:  Star Wars Episode VI: Return of the Jedi
X:  The Masked Gang : Cyprus
y:  The Masked Gang: Cyprus
X:  MPAA Weathering with You
y:  Weathering with You
X:  Good Nei ##bors
y:  Bor
X:  The Princess and the Frog
y:  The Princess and the Frog
X:  Tom Meets Z ##izou
y:  Tom Meets Zizou
X:  X - Men : First Class
y:  X-Men: First Class
X:  Halle Berry
y:  Halle Berry
X:  Julia Roberts
y:  Julia Roberts
X:  Sandra Bullock
y:  Sandra Bullock


#### Try with start-end

In [141]:
acc = 0
all_ents = []
for i, q in enumerate(qs_origin):
    preds = model2_nlp(q, aggregation_strategy="simple")

    start = preds[0]
    end = preds[-1]
    start_id = start['start']
    end_id = end['end']
    print(start_id, end_id)
    all_ent = q[start_id: end_id]
        
        
    
    print("Ground truth: {}".format(Q_ents[i]))
    print("Predicted name entity: {}".format(all_ent))
    print("--------------------------------------------")
    all_ents.append(all_ent)


23 40
Ground truth: Good Will Hunting
Predicted name entity: Good Will Hunting
--------------------------------------------
13 41
Ground truth: The Bridge on the River Kwai
Predicted name entity: The Bridge on the River Kwai
--------------------------------------------
23 65
Ground truth: Star Wars: Episode VI - Return of the Jedi
Predicted name entity: Star Wars: Episode VI - Return of the Jedi
--------------------------------------------
27 50
Ground truth: The Masked Gang: Cyprus
Predicted name entity: The Masked Gang: Cyprus
--------------------------------------------
12 51
Ground truth: Weathering with You
Predicted name entity: MPAA film rating of Weathering with You
--------------------------------------------
21 35
Ground truth: Good Neighbors
Predicted name entity: Good Neighbors
--------------------------------------------
26 51
Ground truth: The Princess and the Frog
Predicted name entity: The Princess and the Frog
--------------------------------------------
40 55
Ground t

In [142]:
all_ents_embeddings = sent_sim.encode(all_ents)
sims = util.pytorch_cos_sim(torch.tensor(all_ents_embeddings), torch.tensor(name_embeddings))
idx = torch.max(sims, 1).indices
for i, id in enumerate(idx):
    print("X: ", all_ents[i])
    print("y: ", names[int(id)])

X:  Good Will Hunting
y:  Good Will Hunting
X:  The Bridge on the River Kwai
y:  The Bridge on the River Kwai
X:  Star Wars: Episode VI - Return of the Jedi
y:  Star Wars: Episode VI – Return of the Jedi
X:  The Masked Gang: Cyprus
y:  The Masked Gang: Cyprus
X:  MPAA film rating of Weathering with You
y:  Weathering with You
X:  Good Neighbors
y:  Good Neighbors
X:  The Princess and the Frog
y:  The Princess and the Frog
X:  Tom Meets Zizou
y:  Tom Meets Zizou
X:  X-Men: First Class
y:  X-Men: First Class
X:  Halle Berry
y:  Halle Berry
X:  Julia Roberts
y:  Julia Roberts
X:  Sandra Bullock
y:  Sandra Bullock


In [199]:
def ner_app():
    pass

def ner_test(X, y, names=names,   name_embeddings=name_embeddings, model_ner=model2_nlp, model_sent_sim=sent_sim, strategy='simple'):

    all_ents = []
    for i, q in enumerate(X):
        preds = model_ner(q, aggregation_strategy=strategy)
        if len(preds) == 0:
            print("---Try model 3---")
            preds = model3_nlp(q, aggregation_strategy=strategy)
            if len(preds) == 0:
                print(q)
                print('No name entity was found.')
                all_ents.append(q)
            else:
                all_ent = q[preds[0]['start']: preds[-1]['end']]
                all_ents.append(all_ent)
        else:
            all_ent = q[preds[0]['start']: preds[-1]['end']]
            # remove substring 'xxx rating'
            all_ents.append(all_ent)

    all_ents_embeddings = model_sent_sim.encode(all_ents)
    sims = util.pytorch_cos_sim(torch.tensor(all_ents_embeddings), torch.tensor(name_embeddings))
    idx = torch.max(sims, 1).indices
    
    ner_pred = all_ents
    y_pred = np.array([names[int(id)] for id in idx])
    y = np.array(y)
    # for i in range(len(y)):
    #     if y_pred[i]!=y[i]:
    #         # print("-----------------------------")
    #         print("Question: {}".format(X[i]))
    #         print("NER prediction: {}".format(ner_pred[i]))
    #         print("NER matching: {}".format(y_pred[i]))
    #         print("Ground truth: {}".format(y[i]))
    #         print("-----------------------------")
    #         # print(X[i], ner_pred[i], y_pred[i], y[i], )
    accuracy = np.mean(y==y_pred)
    return accuracy
    

In [170]:
small_data = pd.read_csv('Datasets/Relations_small_X_y.tsv', sep='\t')


In [ ]:
i, step = 0, 10000
ner_test(list(small_data['X'][i:i+step]), list(small_data['y'][i:i+step]))

In [227]:
def ner_for_recommend(X, y, names=names,   name_embeddings=name_embeddings, model_ner=model2_nlp, model_sent_sim=sent_sim, strategy='simple'):
    X = [
        # "Recommend movies like Nightmare on Elm Street, Friday the 13th, and Halloween.",
        # "Given that I like The Lion King, Pocahontas, and The Beauty and the Beast, can you recommend some movies?",
        "Recommend movies similar to Hamlet and Othello.",
    ]

    all_ents = []
    for i, q in enumerate(X):
        preds = model_ner(q, aggregation_strategy=strategy)
        ents = [pred['word'] for pred in preds ]
        all_ents.append(ents)
    
        ents_embeddings = model_sent_sim.encode(ents)
        sims = util.pytorch_cos_sim(torch.tensor(ents_embeddings), torch.tensor(name_embeddings))
        idx = torch.max(sims, 1).indices
        print(idx)
        for id in np.array(idx):
            print(names[id])

In [1]:
ner_for_recommend([],[])

NameError: name 'ner_for_recommend' is not defined

#### Try with whole sentences

In [ ]:
whole_question_embeddings = sent_sim.encode(qs_origin)

In [68]:
Name_Qwiki_Qid.iloc[list(names).index("Star Wars Episode VI: Return of the Jedi")]

Str      Star Wars Episode VI: Return of the Jedi
Wiki    <http://www.wikidata.org/entity/Q3795592>
Qid                                      Q3795592
Name: 95043, dtype: object

In [86]:
# matchings = []
# for i, embd in enumerate(whole_question_embeddings):
#     results = [util.pytorch_cos_sim(embd, name_embd) for name_embd in name_embeddings]
#     matching = names[results.index(max(results))]
#     matchings.append(matching)
#     print("X: ", qs_origin[i])
#     print("y: ", matching)

X:  Who is the director of Good Will Hunting?
y:  Good Will Hunting
X:  Who directed The Bridge on the River Kwai?
y:  River Kwai bridge
X:  Who is the director of Star Wars: Episode VI - Return of the Jedi?
y:  Star Wars: Episode VI – Return of the Jedi
X:  Who is the screenwriter of The Masked Gang: Cyprus?
y:  The Masked Gang: Cyprus
X:  What is the MPAA film rating of Weathering with You?
y:  Filmiroda rating category
X:  What is the genre of Good Neighbors?
y:  Good Neighbors
X:  What is the box office of The Princess and the Frog?
y:  The Princess and the Frog


KeyboardInterrupt: 

In [ ]:
# acc = 0
# for i, q in enumerate(qs_origin):
#     pred = nlp(q)
#     ent = pred[0]['word']
#     if ent[0] == ' ':
#         ent = ent[1:]
#     print("({})".format(ent), "({})".format(Q_ents[i]))
#     print(ent==Q_ents[i])
#     acc += (ent == Q_ents[i])
# print(acc/len(qs_origin))

# acc = 0
# for i, q in enumerate(qs_origin):
#     pred = model.predict(
#         text=q,
#         generation_kwargs=generation_kwargs,
#         instruction=instruction,
#         options=options,
#     )
#     print(pred[1])
#     ent = q(pred[1][0][0], pred[1][0][1])
#     print("({})".format(ent), "({})".format(Q_ents[i]))
#     print(ent==Q_ents[i])
#     acc += (ent == Q_ents[i])
# print(acc/len(qs_origin))


acc = 0
for i, q in enumerate(qs_origin):
    pred = model2_nlp(q, aggregation_strategy="simple")
    print(pred[0]['word'])
    ent = pred[0]['word']
    print("({})".format(ent), "({})".format(Q_ents[i]))
    print(ent==Q_ents[i])
    acc += (ent == Q_ents[i])
print(acc/len(qs_origin))


### Test for Model 3

In [83]:
acc = 0
all_ents = []
for i, q in enumerate(qs_origin):
    preds = model3_nlp(q, aggregation_strategy='simple')
    ents = []
    tags = []
    # if len(preds) == 0:
    #     return None
    for pred in preds:
        ent = pred['word']
        # tag = pred['entity_group']
        if ent[0] == ' ':
            ent = ent[1:]
        ents.append(ent)
        # tags.append(tag)
    print("--------------------------------------------")
    print("Ground truth: {}".format(Q_ents[i]))
    print("Predicted name entity: ", ents)
    all_ents.append(ents)


--------------------------------------------
Ground truth: Good Will Hunting
Predicted name entity:  ['Good', 'Will Hunting'] ['MISC', 'ORG']
--------------------------------------------
Ground truth: The Bridge on the River Kwai
Predicted name entity:  ['The Bridge on the River K', '##wai'] ['MISC', 'LOC']
--------------------------------------------
Ground truth: Star Wars: Episode VI - Return of the Jedi
Predicted name entity:  ['Star Wars', 'Episode VI', 'Return of the Jedi'] ['MISC', 'MISC', 'MISC']
--------------------------------------------
Ground truth: The Masked Gang: Cyprus
Predicted name entity:  ['The Masked Gang', 'Cyprus'] ['MISC', 'LOC']
--------------------------------------------
Ground truth: Weathering with You
Predicted name entity:  ['MPAA', 'Weathering with You?'] ['ORG', 'MISC']
--------------------------------------------
Ground truth: Good Neighbors
Predicted name entity:  ['Good Neighbors'] ['MISC']
--------------------------------------------
Ground truth: 

几个方向
1. 找最好的ner 模型
2. 看看除了策略还有没有可调参数 比如消除##
3. 自己train/fine-tune： 需要冷启动训练集

### Fuzzy Test (Not useful)

In [88]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [89]:
query = qs_origin[0]
choices = list(names)